In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('data/stage_1_train_with_one_image_dropped.csv')

In [3]:
df_train.head()

,ID,Label
0,ID_63eb1e259_epidural,0
1,ID_63eb1e259_intraparenchymal,0
2,ID_63eb1e259_intraventricular,0
3,ID_63eb1e259_subarachnoid,0
4,ID_63eb1e259_subdural,0


In [4]:
df_train['fn'] = df_train.ID.apply(lambda x: '_'.join(x.split('_')[:2]) + '.png')

In [5]:
df_train.columns = ['ID', 'probability', 'fn']

In [6]:
df_train['label'] = df_train.ID.apply(lambda x: x.split('_')[-1])

In [7]:
df_train.head()

,ID,probability,fn,label
0,ID_63eb1e259_epidural,0,ID_63eb1e259.png,epidural
1,ID_63eb1e259_intraparenchymal,0,ID_63eb1e259.png,intraparenchymal
2,ID_63eb1e259_intraventricular,0,ID_63eb1e259.png,intraventricular
3,ID_63eb1e259_subarachnoid,0,ID_63eb1e259.png,subarachnoid
4,ID_63eb1e259_subdural,0,ID_63eb1e259.png,subdural


In [8]:
df_train.shape

(4045566, 4)

In [9]:
df_train.drop_duplicates('ID', inplace=True)

In [10]:
df_train.shape

(4045542, 4)

In [11]:
pivot = df_train.pivot(index='fn', columns='label', values='probability')

In [12]:
pivot.head()

label,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
fn,,,,,,
ID_000039fa0.png,0,0,0,0,0,0
ID_00005679d.png,0,0,0,0,0,0
ID_00008ce3c.png,0,0,0,0,0,0
ID_0000950d7.png,0,0,0,0,0,0
ID_0000aee4b.png,0,0,0,0,0,0


In [13]:
pivot.reset_index(inplace=True)

In [14]:
pivot.head()

label,fn,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000039fa0.png,0,0,0,0,0,0
1,ID_00005679d.png,0,0,0,0,0,0
2,ID_00008ce3c.png,0,0,0,0,0,0
3,ID_0000950d7.png,0,0,0,0,0,0
4,ID_0000aee4b.png,0,0,0,0,0,0


In [15]:
pivot.shape

(674257, 7)

In [16]:
pivot.to_csv('data/train_pivot.csv', index=False)

The pivoted version of the data can be useful down the road so let's go ahead and save it.

Nonetheless, I just checked the documentation and as we will be using the data_block API for our initial model, we need to reformat the data slightly.

In [17]:
from collections import defaultdict

d = defaultdict(list)
for fn in df_train.fn.unique(): d[fn]

for tup in df_train.itertuples():
    if tup.probability: d[tup.fn].append(tup.label)

In [18]:
ks, vs = [], []

for k, v in d.items():
    ks.append(k), vs.append(' '.join(v))

In [19]:
pd.DataFrame(data={'fn': ks, 'labels': vs}).to_csv('data/train_labels_as_strings.csv', index=False)